In [197]:
import os
import math
from datetime import datetime
from pathlib import Path

import h5py
import numpy as np
from tqdm import tqdm


In [198]:
src_path = "/Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_filtered_20250714.h5"
dst_path = "/Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714.h5"

In [199]:
if "src" in vars():
    src.close()   # type: ignore

src = h5py.File(src_path, "r")
for attr in src.attrs:
    print(f"{attr}: {src.attrs[attr]}")

author: Mark Vodyanitskiy (mvodya@icloud.com)
created_at: 2025-07-13T14:26:08.378871
filter_rules: MIN_TOTAL_POINTS=50, MIN_MOVING_POINTS=5, MIN_MAX_SPEED=20, SPEED_MOVING_MIN=10, SPEED_SANITY_MAX=800
filtered_at: 2026-01-08T06:02:47.931391
sources_count: 27555
sources_size: 439.3Gb
version: 1.0


In [ ]:
SPEED_MOVING_MIN = 10            # 1.0 knot (speed in tenths)
STOP_RADIUS_M = 250.0            # стоим "на месте"
STOP_DWELL_SEC = 30 * 60         # 30 минут в стопе => конец трека

GAP_HARD_SEC = 5 * 3600          # "большой" разрыв времени
GAP_VERY_HARD_SEC = 10 * 3600    # "огромный" разрыв => всегда новый трек
DIST_AFTER_GAP_M = 150_000.0     # после разрыва появилось > 150 км от места => новый трек

JUMP_HARD_M = 250_000.0          # внезапный скачок координат => новый трек

DEST_GAP_SEC = 4 * 3600          # destination сменился и был разрыв > 4ч
DEST_DIST_M = 50_000.0           # destination сменился и мы уехали далеко

In [201]:
def _haversine_m(lat1, lon1, lat2, lon2):
    # Быстрая (и достаточно точная) haversine в метрах
    # lat/lon в градусах
    r = 6371000.0
    p = math.pi / 180.0
    dlat = (lat2 - lat1) * p
    dlon = (lon2 - lon1) * p
    a = math.sin(dlat * 0.5) ** 2 + math.cos(lat1 * p) * math.cos(lat2 * p) * math.sin(dlon * 0.5) ** 2
    c = 2.0 * math.asin(min(1.0, math.sqrt(a)))
    return r * c


def _decode_bytes(x):
    # destination / name и т.п. могут быть np.bytes_
    if isinstance(x, (bytes, np.bytes_)):
        return bytes(x)
    return x


def iter_day_datasets(src_h5):
    # Возвращает список (yyyymmdd_key, dataset) отсортированный по дате
    # key удобен только для сортировки и tqdm
    out = []
    pos = src_h5["positions"]
    for y in sorted(pos.keys()):
        gy = pos[y]
        for m in sorted(gy.keys()):
            gm = gy[m]
            for d in sorted(gm.keys()):
                ds = gm[d]
                out.append((f"{y}-{m}-{d}", ds))
    return out

IDLE_TRACK_ID = -1  # что писать в positions.track_id, пока судно "стоит после завершенного трека"

def _set_idle(st, ts, lat, lon, dest):
    st["idle"] = True
    st["idle_since_ts"] = int(ts)
    st["idle_lat"] = float(lat)
    st["idle_lon"] = float(lon)
    st["idle_dest"] = dest

    # стоп-логика остается активной
    st["stop_active"] = True
    st["stop_anchor_lat"] = float(lat)
    st["stop_anchor_lon"] = float(lon)
    st["stop_accum_sec"] = 0

def _idle_should_start(st, ts, lat, lon, speed, dest):
    # Выходим из IDLE, когда пошло устойчивое движение/смещение.
    # 1) скорость >= порога
    # 2) или заметное смещение от точки стоянки (на случай кривой speed)
    sp = int(speed)
    moved_far = _haversine_m(st["idle_lat"], st["idle_lon"], float(lat), float(lon)) > STOP_RADIUS_M
    dest_changed = (dest != st.get("idle_dest", None))
    return (sp >= SPEED_MOVING_MIN and moved_far) or (sp >= SPEED_MOVING_MIN and dest_changed) or (moved_far and dest_changed)

In [ ]:
# Dtypes возьмем из исходника (важно, чтобы track_id поле уже было в positions dtype)
ships_dtype = src["ships"].dtype
files_dtype = src["files"].dtype
tracks_dtype = src["tracks"].dtype
zones_dtype = src["zones"].dtype if "zones" in src else None

dst = h5py.File(dst_path, "w")

# Атрибуты исходника
for k, v in src.attrs.items():
    dst.attrs[k] = v

# Атрибуты про построение треков
dst.attrs["tracks_built_at"] = datetime.utcnow().isoformat()
dst.attrs["tracks_rules"] = (
    f"SPEED_MOVING_MIN={SPEED_MOVING_MIN}, STOP_RADIUS_M={STOP_RADIUS_M}, STOP_DWELL_SEC={STOP_DWELL_SEC}, "
    f"GAP_HARD_SEC={GAP_HARD_SEC}, GAP_VERY_HARD_SEC={GAP_VERY_HARD_SEC}, DIST_AFTER_GAP_M={DIST_AFTER_GAP_M}, "
    f"JUMP_HARD_M={JUMP_HARD_M}, DEST_GAP_SEC={DEST_GAP_SEC}, DEST_DIST_M={DEST_DIST_M}"
)

# Копируем плоские таблицы целиком (ships/files/zones)
dst.create_dataset("ships", data=src["ships"][:], dtype=ships_dtype,
                   chunks=True, compression="gzip", compression_opts=4)

dst.create_dataset("files", data=src["files"][:], dtype=files_dtype,
                   chunks=True, compression="gzip", compression_opts=4)

if "zones" in src:
    dst.create_dataset("zones", data=src["zones"][:], dtype=zones_dtype,
                       chunks=True, compression="gzip", compression_opts=4)

# Создаем пустой /tracks (resizable)
dst.create_dataset("tracks", shape=(0,), maxshape=(None,), dtype=tracks_dtype,
                   chunks=True, compression="gzip", compression_opts=4)

# Создаем группу positions (дальше будем создавать year/month/day datasets)
dst.create_group("positions")

print("dst created:", dst_path)


/var/folders/2c/2p1h64qd4zb1g13m_f5z4z9w0000gn/T/ipykernel_56656/2673037174.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  dst.attrs["tracks_built_at"] = datetime.utcnow().isoformat()


dst created: /Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714.h5


In [ ]:
# ----------------------------
# State per ship
# ----------------------------
# ship_state[ship_id] = dict with:
#   track_id, start_ts, start_lat, start_lon
#   last_ts, last_lat, last_lon, last_speed, last_course, last_dest
#   points_count
#   stop_accum_sec, stop_anchor_lat, stop_anchor_lon, stop_active
ship_state = {}

next_track_id = 1

def _append_track(dst_tracks, row):
    n = dst_tracks.shape[0]
    dst_tracks.resize((n + 1,))
    dst_tracks[n] = row

def _close_track_if_any(dst_tracks, st):
    if st.get("idle", False):
        return
    if st["points_count"] < 2:
        return
    # Закрываем текущий трек состояния st
    # Требуем минимум 2 точки (иначе трек мусор)
    if st["points_count"] < 2:
        return
    row = np.zeros((1,), dtype=dst_tracks.dtype)[0]
    row["track_id"] = st["track_id"]
    row["ship_id"] = st["ship_id"]
    row["start_timestamp"] = st["start_ts"]
    row["end_timestamp"] = st["last_ts"]
    row["start_lat"] = st["start_lat"]
    row["start_lon"] = st["start_lon"]
    row["end_lat"] = st["last_lat"]
    row["end_lon"] = st["last_lon"]
    row["points_count"] = st["points_count"]
    _append_track(dst_tracks, row)

def _start_new_track(st, ship_id, ts, lat, lon, speed, course, dest):
    global next_track_id
    st["idle"] = False
    st["track_id"] = next_track_id
    next_track_id += 1

    st["ship_id"] = ship_id
    st["start_ts"] = ts
    st["start_lat"] = float(lat)
    st["start_lon"] = float(lon)

    st["last_ts"] = ts
    st["last_lat"] = float(lat)
    st["last_lon"] = float(lon)
    st["last_speed"] = int(speed)
    st["last_course"] = int(course)
    st["last_dest"] = dest

    st["points_count"] = 1

    st["stop_accum_sec"] = 0
    st["stop_anchor_lat"] = float(lat)
    st["stop_anchor_lon"] = float(lon)
    st["stop_active"] = (int(speed) < SPEED_MOVING_MIN)

def _need_new_track(st, ts, lat, lon, speed, course, dest):
    last_ts = st["last_ts"]
    dt = int(ts) - int(last_ts)
    if dt < 0:
        dt = 0

    dist = _haversine_m(st["last_lat"], st["last_lon"], float(lat), float(lon))

    last_dest = st["last_dest"]
    dest_changed = (dest != last_dest)

    # 1) Очень большой разрыв времени
    if dt > GAP_VERY_HARD_SEC:
        return True, dt, dist, dest_changed

    # 2) Разрыв + заметное смещение
    if dt > GAP_HARD_SEC and dist > DIST_AFTER_GAP_M:
        return True, dt, dist, dest_changed

    # 3) Большой скачок координат
    if dist > JUMP_HARD_M:
        return True, dt, dist, dest_changed

    # 4) destination сменился + подтверждение
    if dest_changed:
        if dt > DEST_GAP_SEC or dist > DEST_DIST_M or st["stop_active"]:
            return True, dt, dist, dest_changed

    return False, dt, dist, dest_changed


def _update_stop_logic(st, dt, dist, speed, lat, lon):
    # Обновляем "стоп": судно считается стоящим, если speed < SPEED_MOVING_MIN и в радиусе STOP_RADIUS_M
    sp = int(speed)
    if sp < SPEED_MOVING_MIN:
        # если уже стоп - якорь фиксированный, иначе ставим якорь текущей точкой
        if not st["stop_active"]:
            st["stop_active"] = True
            st["stop_anchor_lat"] = float(lat)
            st["stop_anchor_lon"] = float(lon)
            st["stop_accum_sec"] = 0

        # проверим, что остаемся в радиусе якоря
        d_anchor = _haversine_m(st["stop_anchor_lat"], st["stop_anchor_lon"], float(lat), float(lon))
        if d_anchor <= STOP_RADIUS_M:
            st["stop_accum_sec"] += int(dt)
        else:
            # "поползли" - перезаякоримся
            st["stop_anchor_lat"] = float(lat)
            st["stop_anchor_lon"] = float(lon)
            st["stop_accum_sec"] = 0
    else:
        # движется - стоп сбрасываем
        st["stop_active"] = False
        st["stop_accum_sec"] = 0


# Prepare tqdm totals (metadata scan only)
day_list = iter_day_datasets(src)
total_positions = sum(ds.shape[0] for _, ds in day_list)

pbar = tqdm(total=total_positions, unit="pos", desc="Build tracks")

# Main loop over days
CHUNK_ROWS = 2_000_000  # подстрой под RAM/скорость

for day_key, src_day in day_list:
    # Создадим соответствующий путь в dst: /positions/YYYY/MM/DD
    # day_key = "YYYY-MM-DD"
    y, m, d = day_key.split("-")
    gpos = dst["positions"]
    if y not in gpos:
        gpos.create_group(y)
    if m not in gpos[y]:
        gpos[y].create_group(m)

    # Создаем dst dataset для дня
    nrows = src_day.shape[0]
    dst_day = gpos[y][m].create_dataset(
        d,
        shape=(nrows,),
        maxshape=(nrows,),
        dtype=src_day.dtype,
        chunks=True,
        compression="gzip",
        compression_opts=4,
    )

    # Поля как массивы (ускоряет доступ в цикле)
    for start in range(0, nrows, CHUNK_ROWS):
        end = min(nrows, start + CHUNK_ROWS)
        chunk = src_day[start:end]  # numpy structured array

        ship_ids = chunk["ship_id"]
        ts_arr = chunk["timestamp"]
        lat_arr = chunk["lat"]
        lon_arr = chunk["lon"]
        spd_arr = chunk["speed"]
        crs_arr = chunk["course"]
        dest_arr = chunk["destination"]

        # Результат пишем сюда же (копия чанка, чтобы менять track_id)
        out = chunk.copy()

        for i in range(out.shape[0]):
            ship_id = int(ship_ids[i])
            ts = int(ts_arr[i])
            lat = float(lat_arr[i])
            lon = float(lon_arr[i])
            speed = int(spd_arr[i])
            course = int(crs_arr[i])
            dest = _decode_bytes(dest_arr[i])

            st = ship_state.get(ship_id)
            if st is None:
                st = {}
                ship_state[ship_id] = st
                _start_new_track(st, ship_id, ts, lat, lon, speed, course, dest)
                out["track_id"][i] = st["track_id"]
                continue

            new_track, dt, dist, dest_changed = _need_new_track(st, ts, lat, lon, speed, course, dest)

            # Логика долго стояли => конец трека
            _update_stop_logic(st, dt, dist, speed, lat, lon)
            stop_long_enough = st["stop_active"] and st["stop_accum_sec"] >= STOP_DWELL_SEC

            # Если уже в IDLE - не создаем трек, пока не началось движение
            if st.get("idle", False):
                if _idle_should_start(st, ts, lat, lon, speed, dest):
                    _start_new_track(st, ship_id, ts, lat, lon, speed, course, dest)
                    out["track_id"][i] = st["track_id"]
                else:
                    out["track_id"][i] = IDLE_TRACK_ID
                    # обновим якорь для стоянки (чтобы stop_accum_sec копился корректно)
                    # dt тут можно брать из вычисленного ранее
                    _update_stop_logic(st, dt, 0.0, speed, lat, lon)
                    st["last_ts"] = ts
                    st["last_lat"] = lat
                    st["last_lon"] = lon
                    st["last_speed"] = speed
                    st["last_course"] = course
                    st["last_dest"] = dest
                continue

            # Обычный режим (не IDLE)
            if new_track:
                _close_track_if_any(dst["tracks"], st)
                _start_new_track(st, ship_id, ts, lat, lon, speed, course, dest)
                out["track_id"][i] = st["track_id"]
                continue

            if stop_long_enough:
                # Закрываем трек и уходим в IDLE, на текущую точку ставим -1
                _close_track_if_any(dst["tracks"], st)
                _set_idle(st, ts, lat, lon, dest)
                out["track_id"][i] = IDLE_TRACK_ID

                # обновим last_* чтобы dt считался от последней точки
                st["last_ts"] = ts
                st["last_lat"] = lat
                st["last_lon"] = lon
                st["last_speed"] = speed
                st["last_course"] = course
                st["last_dest"] = dest
                st["points_count"] = 0  # чтобы случайно не закрывать пустой трек повторно
                continue

            # Продолжаем текущий трек
            out["track_id"][i] = st["track_id"]
            st["last_ts"] = ts
            st["last_lat"] = lat
            st["last_lon"] = lon
            st["last_speed"] = speed
            st["last_course"] = course
            st["last_dest"] = dest
            st["points_count"] += 1

        dst_day[start:end] = out
        pbar.update(end - start)

pbar.close()

# Закрываем все открытые треки
for st in ship_state.values():
    _close_track_if_any(dst["tracks"], st)

dst.attrs["tracks_count"] = int(dst["tracks"].shape[0])

print("tracks:", dst["tracks"].shape[0])


Build tracks: 100%|██████████| 826329360/826329360 [2:03:03<00:00, 111915.79pos/s]  


tracks: 11731643


In [204]:
dst.flush()
dst.close()
src.close()
print("done:", dst_path)


done: /Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_tracks_20250714.h5
